(work progress draft, but it works)
- add chunk overlapping!
- add period edge cases like Dr. is not the end of a sentence.

# Training & Retrieval Material: Personal Data Set Creation
A significant problem and gap between public-mega-model AI and private real-world-use is that a combination of private and copyrighted material comprise a large part of what private individuals and groups need to use AI for, and by definition this is what public-mega-ai is prohibited from being able to help with (sometimes for good reasons).

One way of looking at this is the Clear-Web vs. Deep-Web gap which becomes semantially elusive. When people say 'the internet' or 'the whole web' it sounds like they are talking about most of the material in the world, or at least 'the whole internet' but as they say: Naming things is hard. The entire-public-internet is not the same as the-entire-internet. Training on 'the entire public internet' actually means about 4% of the internet which is the public "clear-web." The rest of the internet is the other 95% of the internet (or Deep-web) and is not public, let alone all the material not on the internet at all (which for many institutions is most of their files). (And then about 0.5% is the dark web...but we're not sure how big the dark web is because it's so hard to see any of it at all.)

To help square this circle, the proposed method here is to make available a pipeline by which people can turn their private or personally owned documents into formatted Training & Retreival Material which can be used to train, fine-tune, and make databases for retrieval augumented uses of AI (which may be generative or more than just generative). (No doubt there are other very similar projects.)

# Smart chunking

python code

a system in python to ingest an epub book into a .txt file

code in python to 'chunk' this text with the following policies and rules.

1. redundancy/overlap between chunks (no system yet, may vary widely by context of use, e.g. RAG or fine tune)
2. smart-chunking where the chunk will not be cut in random places (cutting words or phrases in half)
3. try to identify paragraph ends to make a cut (not done yet)
4. try to identify sentence ends to make a cut (currrent norm)
5. specifically check that the word 'not' is not isolated at or within ~25 characters to either side of a cut (so that negative meanings are not invented or lost to redacted context) (should be covered by sentence-based cuts.)
6. specified chunk max size (defaults to 500 char, set to any input)
7. specified overlap size
8. method of padding for exact chunk size (not done yet)
9. look at how code and other non-sentence material is cut (not done yet)


iterate through chapter files...
maybe break up text if the text is too large...issue.



When files are too long to chunk(by the first method), and so need to be chunked (by another method):

So perhaps if a file/string's length is above say, 10,000 characters.

Steps:
1. Look at a range of ~500 characters the middle of the file loaded as a string: e.g.
```
middle_section_start = string[len)//2 - 250
middle_section_end = string[len)//2 + 250
string[middle_section_start:middle_section_end]
```

Within that section, look for the end of a paragraph or end of a sentence.

". " or ".\n"

find the first likely end of a sentence of paragraph in that
middle of a block of text.


try to avoid sub-sences that are intside quotes or parenthesis
"yyy."
(yyy?)

What to do when a sentence itself is more than the chunk size...

for freak-sentences that are longer then 500 char...
maybe redundantly splits the sentences avoiding mid-word and 'not' cuts




# Metadata:

## Maybe Goal:
Generate a directory of metadata per chunk,
so that from this menue of meta-data specific ~versions of the chunk can be created,
e.g.
- for graph-db
- for vector-db
- for formats of training-data:
-- instruct
-- Q&A
-- dialogue
-- raw input
-- augmented input(s)

Project:
epub should be an excellent format for providing specific metadata for chunks of the text, as there is an index, a table of contents, and often information about where the chunk exists in the content structure.

Structural Mapping:  Analyze the EPUB's structure (table of contents, chapters, sections, index) to locate relevant text chunks and establish their hierarchy.

metadata may be used (or designed specifically for):
- vector DB RAG
- graph DB RAG
- Q&A format model training / fine tuning dataset
- instruction based model training / fine tuning dataset
- maybe text with meta-data context for simple text model training / fine tuning dataset

possible metatdata areas
summary:
example question:
name of text:
name of chapter/section:
topic of chapter/section:
topic links from book index:

Tasks:
- make a Metadata Schema
- have parameters for what kinds of metadata are created

- focus on the initial task of what meta-data can be derived directly from the epub files such as table of contents(at the start) and index (at the end) both of which have hyperlinks to connect text and terms.


Note: chunk size may need to shrinkk if chunk-metadata is

Why Structural Metadata Matters

Enhanced Navigation: TOC and index serve as crucial tools for readers to quickly navigate within a complex document and locate specific information.
Improved Discoverability: Index entries and keywords enhance the visibility of your EPUB within search engines and digital libraries.
Semantic Structure: Structural metadata adds a layer of semantic information to the text, improving machine understanding of the document content. This is valuable for the kinds of AI applications you mentioned earlier.

### Metadata Schema Design


## Specific Sources:
- Table of Contents (TOC): A hierarchical structure representing the document's layout, including chapters, sections, and subsections.
- (sometimes) detailed table of contents
- Index: A list of terms, topics, and names with hyperlinks to their occurrences in the document.
- Chapter/Section introductions


## 1. Basic Metadata Items:
- Title: The name of the EPUB document.
- Author(s): The creator(s) of the document.
- Publisher: The publisher of the document.
- Edition
- Publication Date: When the document was published.
- Language: The primary language of the document.
- ISBN: The International Standard Book Number, if available.

## 2. Structural Metadata Items:
- section_name: The title of the chapter or section.
- ID/Reference: A unique identifier or reference within the document.
- Topic(s): Main topics covered in the chapter or section.
- Keywords: A list of key terms associated with the chapter or section.

## 3. Content-Specific Metadata
- Summary: A brief overview of the document or specific sections/chapters.
- Example Questions: Potential questions that the content can answer, useful for Q&A model training.
- Topic Links: Hyperlinks within the index that connect to specific topics, terms, or sections in the document.

## 4. Advanced Metadata for Machine Learning
- Vector Representations: Metadata that includes vector representations of chapters or sections, useful for vector databases and similarity searches.
- Graph Connections: Information on how different sections are related or can be linked, valuable for graph databases.
- Instruction Metadata: Specific instructions or tasks that can be derived from the content, useful for instruction-based model training.

# challenges

1. RAG DB: metadata for graph-DB and for vector-DB are very different in their use and requirements

2. Size: if metadata, or some metadata, is included in the chunk, that will shrink the chunk itself. E.g. max 500 char = 200 char of metadata + 300 char of actual document text

3. pragmatism: the low hanging fruit of epub data extraction is not clearly
useful:
```
def extract_global_metadata(opf_content):
    tree = ET.ElementTree(ET.fromstring(opf_content))
    root = tree.getroot()
    ns = {'dc': 'http://purl.org/dc/elements/1.1/'}
    
    global_metadata = {
        'title': root.find('.//dc:title', ns).text,
        'author': root.find('.//dc:creator', ns).text,
        'publisher': root.find('.//dc:publisher', ns).text,
        'publication_date': root.find('.//dc:date', ns).text,
        'language': root.find('.//dc:language', ns).text
    }
    
    return global_metadata
```
```
def append_chunks_to_jsonl(chunks_list, output_chunks_jsonl_path, chunk_source_name, metadata):
    with open(output_chunks_jsonl_path, 'a') as f:
        for index, this_chunk in enumerate(chunks_list):
            chunk_data = {
                "source_name": f"{chunk_source_name}_{index}",
                "text": this_chunk,
                "metadata": metadata  # Include the metadata here
            }
            f.write(json.dumps(chunk_data) + '\n')

```

# TODO chat-ML formatting for dataset processing of chunked jsonl with metadata

e.g.

terms from index

chunk:
author:
book name:
section/chapter:
index key words:

user: please tell me more about {if fields: "including {index key words}"}} in {author's}{book title} section {chapter/section}

assistant: In {chunk}

...

This comes from the example of using color descriptions to make a 'color expert' chatbot

# Task:
- looking up index terms
- possibly connecting sections to other sections (real-graph? or vector-gossip-graph?)


To Jan:

Feature Request: allow epub books

Hopefully code like this (see most recent version)
https://github.com/lineality/epub_ingestion_python/blob/main/epub_injestion_jsonl_txt_sized_chunks_v21.py

Will be useful in letting users of Jan use epub-books with their Jan Retrieval function.

This code extracts text from epub books into a variety of formats, txt json, jsonl, and can chunk to specific sizes without cutting words or sentences in half, to better retain meaning.

In [ ]:
#########################################################
# This code automaticaly finds and processes epub books
# esspecially for RAG document ingestion processing
#########################################################
"""
input -> one or more epub files
output -> txt and json files that contain the text from sections of the book
          as well as smart-chunked segments made to your size specs
          e.g. a max of 500 characters, which contain whole sentences.
          Chunks do not cut words or sentences in half.

# Set of Results, saved in a file per epub doc:
1. One .jsonl file
2. (Plural) Individual .json files in a folder
3. One .txt text file of the whole epub
4. (plural) Individual .txt files from separate parts of epub
5. chunks as one .jsonl file
6. (Plural) chunks under specified character length as separate text
Future Feature:
7. Chunk-Metadata (for model training, for DB-retrieval, etc.)
"""

import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import json
import os
import shutil
import re

def get_ordered_html_files(opf_content):
    """
    Parses the content.opf file to determine the reading order of HTML files in the EPUB.

    The function reads the 'content.opf' file, which contains metadata about the EPUB's structure.
    It identifies the 'spine' element, which lists the reading order of the content documents,
    and the 'manifest' element, which provides the location of these documents.
    The function returns a list of HTML file paths in the order they should be read.

    Args:
    opf_content (str): A string representation of the content.opf file.

    Returns:
    list: An ordered list of HTML file paths as specified in the EPUB's spine.
    """

    # Parse the content.opf XML content
    tree = ET.ElementTree(ET.fromstring(opf_content))
    root = tree.getroot()

    # Define the namespace for the OPF package file
    ns = {'opf': 'http://www.idpf.org/2007/opf'}

    # Find the spine element which indicates the order of the content documents
    spine = root.find('opf:spine', ns)
    itemrefs = spine.findall('opf:itemref', ns)

    # Extract the id references for each item in the spine
    item_ids = [itemref.get('idref') for itemref in itemrefs]

    # Find the manifest element which lists all the content documents
    manifest = root.find('opf:manifest', ns)
    items = manifest.findall('opf:item', ns)

    # Create a dictionary mapping item IDs to their corresponding file paths
    html_files = {item.get('id'): item.get('href') for item in items if item.get('media-type') == 'application/xhtml+xml'}

    # Generate an ordered list of HTML files based on the spine order
    ordered_html_files = [html_files[item_id] for item_id in item_ids if item_id in html_files]

    return ordered_html_files


def extract_text_from_html(html_content, this_epub_output_dir_path):
    """
    Extracts and returns text from an HTML content.
    """
    # print("HTML Content before BeautifulSoup Parsing:\n", html_content[:500])  # Print first 500 characters of HTML
    print_and_log(f"\nlen(HTML Content before BeautifulSoup Parsing) -> {len(html_content)}", this_epub_output_dir_path)  # Print first 500 characters of HTML


    soup = BeautifulSoup(html_content, 'html.parser')
    parsed_text = soup.get_text()
    # print("Extracted Text:\n", parsed_text[:500])  # Print first 500 characters of extracted text
    print_and_log(f"len(Extracted Text) -> {len(parsed_text)}", this_epub_output_dir_path)  # Print first 500 characters of extracted text

    return parsed_text

def fix_text_formatting(text):
    """Replaces the Unicode right single quotation mark with a standard apostrophe."""
    return text.replace("\u2019", "'")


def check_len_chunks_in_list(chunks_list, max_chunk_size, this_epub_output_dir_path):

    size_flag_ok = True

    for index, this_chunk in enumerate(chunks_list):

        # get size of chunk
        this_length = len( this_chunk )

        # check size against user-input max size
        if this_length > max_chunk_size:
            print_and_log( this_length, this_epub_output_dir_path )
            print_and_log( f"""
            Warning: chunk over max size.
            This chunk size: {this_chunk}.
            Max size: {max_chunk_size}
            """ )
            print_and_log( f"This chunk: {this_chunk}", this_epub_output_dir_path )
            size_flag_ok = False

    # report and log
    if size_flag_ok:
        print_and_log( "Size Check, OK \\o/", this_epub_output_dir_path )
    else:
        print_and_log( "WARNING: Size Check Failed!", this_epub_output_dir_path )



def save_individual_chunks(chunks_list, output_chunks_dir, chunk_source_name):

    for index, this_chunk in enumerate(chunks_list):
        chunk_name = f"{chunk_source_name}_{index}.txt"

        # Save individual txt files
        individual_chunk_path = os.path.join(output_chunks_dir, chunk_name)
        with open(individual_chunk_path, 'w') as f:
            f.write(this_chunk)
            # print('chunk writen', individual_chunk_path)

    return len(chunks_list)


def append_chunks_to_jsonl(chunks_list, output_chunks_jsonl_path, chunk_source_name):
    """Appends chunks of text to a .jsonl file, each chunk as a JSON object.

    Args:
        chunks_list (list): List of text chunks to be appended.
        output_jsonl_path (str): The output file path for the .jsonl file.
        chunk_source_name (str): Base name for each chunk, used in the 'source_name' field.

    Returns:
        int: The number of chunks appended.
    """

    with open(output_chunks_jsonl_path, 'a') as f:  # Open file in append mode
        for index, this_chunk in enumerate(chunks_list):
            # Construct a JSON object for the chunk
            chunk_data = {
                "source_name": f"{chunk_source_name}_{index}",
                "text": this_chunk
            }

            # Convert the chunk data to a JSON string and append it to the file with a newline
            f.write(json.dumps(chunk_data) + '\n')

    return len(chunks_list)


def print_and_log(input_text, this_epub_output_dir_path):
    # check if input is a string, if not...make it a string!
    if not isinstance(input_text, str):
        input_text = str(input_text)

    # print to terminal
    print(input_text)

    # log file path is...
    log_file_path = os.path.join(this_epub_output_dir_path, "log.txt")

    # log: Write/Append to a log.txt file
    with open(log_file_path, 'a') as f:
        f.write(input_text + '\n\n')


def extract_text_from_epub(epub_file_path, this_epub_output_dir_path, output_jsonl_path, output_json_dir, output_whole_txt_path, output_txt_dir, output_chunks_jsonl_path, output_chunks_dir, max_chunk_size=500):
    with zipfile.ZipFile(epub_file_path, 'r') as epub:
        print_and_log(f"EPUB Contents: -> {epub.namelist()}", this_epub_output_dir_path)

        ###################
        # Make Directories
        ###################

        # Create a directory for individual JSON files
        if not os.path.exists(output_json_dir):
            os.makedirs(output_json_dir)

        # Create a directory for individual txt files
        if not os.path.exists(output_txt_dir):
            os.makedirs(output_txt_dir)

        # Create a directory for chunks output_chunks_dir
        if not os.path.exists(output_chunks_dir):
            os.makedirs(output_chunks_dir)


        ##################################
        # Get & Read html files from epub
        ##################################
        # find opf file
        opf_file = [f for f in epub.namelist() if 'content.opf' in f][0]

        # read opf file
        opf_content = epub.read(opf_file).decode('utf-8')

        # get ordered HTML files
        ordered_html_files_list = get_ordered_html_files(opf_content)


        ############################################
        # Read and extract text from each HTML file
        ############################################

        # iterate through html files
        for html_file in ordered_html_files_list:
            full_path = os.path.join(os.path.dirname(opf_file), html_file)
            if full_path in epub.namelist():
                html_content = epub.read(full_path).decode('utf-8')

                #########################
                # extract text from epub
                #########################
                raw_text = extract_text_from_html(html_content, this_epub_output_dir_path)
                print_and_log(f"len(text for json)-> {len(raw_text)}", this_epub_output_dir_path)

                # fix text formatting
                text = fix_text_formatting(raw_text)


                #################
                # .json & .jsonl
                #################

                # Write/Append to a single JSONL file
                with open(output_jsonl_path, 'a') as f:
                    json_record = json.dumps({'text': text.strip()})
                    f.write(json_record + '\n')

                # Save individual JSON file
                individual_json_path = os.path.join(output_json_dir, f"{os.path.splitext(html_file)[0]}.json")
                with open(individual_json_path, 'w') as f:
                    json.dump({'text': text.strip()}, f, indent=4)

                #######
                # .txt
                #######

                # Write/Append to a single text .txt file
                with open(output_whole_txt_path, 'a') as f:
                    f.write(text + '\n\n')

                # Save individual txt files
                individual_txt_path = os.path.join(output_txt_dir, f"{os.path.splitext(html_file)[0]}.txt")
                with open(individual_txt_path, 'w') as f:
                    f.write(text)

                #########
                # chunks
                #########

                chunks_list = make_chunk_list(text, max_chunk_size, this_epub_output_dir_path)

                chunk_source_name = os.path.splitext(html_file)[0]

                # check sizes
                check_len_chunks_in_list(chunks_list, max_chunk_size, this_epub_output_dir_path)

                number_of_chunks = save_individual_chunks(chunks_list, output_chunks_dir, chunk_source_name)
                print_and_log(f"Chunked: split into this many chunks-> {number_of_chunks}", this_epub_output_dir_path)

                append_chunks_to_jsonl(chunks_list, output_chunks_jsonl_path, chunk_source_name)

                print_and_log(f"{html_file} -> ok!", this_epub_output_dir_path)


            else:  # File Not Found
                print_and_log(f"Warning: File {full_path} not found in the archive.")



def extract_text_from_txt(txt_file_path, this_txt_output_dir_path, output_jsonl_path, output_json_dir, output_whole_txt_path, output_txt_dir, output_chunks_jsonl_path, output_chunks_dir, max_chunk_size=500):

    # Open the file in read mode
    with open(txt_file_path, 'r') as file:

        ###################
        # Make Directories
        ###################

        # Create a directory for individual JSON files
        if not os.path.exists(output_json_dir):
            os.makedirs(output_json_dir)

        # Create a directory for individual txt files
        if not os.path.exists(output_txt_dir):
            os.makedirs(output_txt_dir)

        # Create a directory for chunks output_chunks_dir
        if not os.path.exists(output_chunks_dir):
            os.makedirs(output_chunks_dir)



        #########################
        # extract text from epub
        #########################
        # Read the entire contents of the file
        text = file.read()

        #################
        # .json & .jsonl
        #################

        # Write/Append to a single JSONL file
        with open(output_jsonl_path, 'a') as f:
            json_record = json.dumps({'text': text.strip()})
            f.write(json_record + '\n')

        # Save individual JSON file
        individual_json_path = os.path.join(output_json_dir, f"{os.path.splitext(txt_file_path)[0]}.json")
        with open(individual_json_path, 'w') as f:
            json.dump({'text': text.strip()}, f, indent=4)

        #######
        # .txt
        #######

        # Write/Append to a single text .txt file
        with open(output_whole_txt_path, 'a') as f:
            f.write(text + '\n\n')

        # Save individual txt files
        individual_txt_path = os.path.join(output_txt_dir, f"{os.path.splitext(txt_file_path)[0]}.txt")
        with open(individual_txt_path, 'w') as f:
            f.write(text)


        #########
        # chunks
        #########

        chunks_list = make_chunk_list(text, max_chunk_size, this_txt_output_dir_path)

        chunk_source_name = os.path.splitext(txt_file_path)[0]

        # check sizes
        check_len_chunks_in_list(chunks_list, max_chunk_size, this_txt_output_dir_path)

        number_of_chunks = save_individual_chunks(chunks_list, output_chunks_dir, chunk_source_name)
        print_and_log(f"Chunked: split into this many chunks-> {number_of_chunks}", this_txt_output_dir_path)

        append_chunks_to_jsonl(chunks_list, output_chunks_jsonl_path, chunk_source_name)

        print_and_log(f"{txt_file_path} -> ok!", this_txt_output_dir_path)

        print("OK!")




def zip_folder(path_to_directory_to_zip, output_destination_zip_file_path):
    """Creates a zip archive of a specified folder.

    Args:
        path_to_directory_to_zip (str): The path to the folder to be zipped.
        output_destination_zip_file_path (str): The desired name and path of the output zip file.
    """
    # # Specify the folder you want to zip
    # path_to_directory_to_zip = "individual_jsons"

    # # Specify the desired output zip file name (e.g., 'jsons_archive.zip')
    # output_destination_zip_file_path = "jsons_archive_zip"

    shutil.make_archive(output_destination_zip_file_path, 'zip', path_to_directory_to_zip)


###########
# Chunking
############


def split_sentences_and_punctuation(text):
    """Splits text into sentences, attempting to preserve punctuation and all text content.
    Args:
        text (str): The input text.
    Returns:
        list: A list of sentences with preserved punctuation.
    """

    ABBREVIATIONS = [
    "Dr.", "Mr.", "Mrs.", "Ms.", "Lt.", "St.", "Capt.", "Col.", "Gen.", "Rev.", "Hon.",
    "Jr.", "Sr.", "Ph.D.", "M.D.", "B.A.", "M.A.", "D.D.S.", "J.D.", "M.B.A.", "C.P.A.",
    "a.m.", "p.m.", "etc.", "i.e.", "e.g.", "vs.", "et al.", "approx.", "circa", "fig.",
    "Jan.", "Feb.", "Mar.", "Apr.", "May.", "Jun.", "Jul.", "Aug.", "Sep.", "Oct.", "Nov.", "Dec.",
    "Prof.", "Assoc.", "Asst.", "Dept.", "Univ.", "Gov.", "Sen.", "Rep.", "Pres.",
    "Vol.", "No.", "Pt.", "Ch.", "Sec.", "Subsec.", "Para.", "Subpara.", "Cl.", "Subcl.",
    "Ex.", "Exs.", "Fig.", "Figs.", "Eq.", "Eqs.", "Thm.", "Thms.", "Prop.", "Props.",
    "Def.", "Defs.", "Cor.", "Cors.", "Lem.", "Lems.", "Proof.", "Proofs.",
    "min.", "hrs.", "sec.", "yr.", "yrs.", "mo.", "mos.", "wk.", "wks.",
    "ft.", "in.", "cm.", "mm.", "kg.", "g.", "oz.", "lb.", "lbs.", "ml.", "l.",
    "A.", "B.", "C.", "D.", "E.", "F.", "G.", "H.", "I.", "J.", "K.", "L.", "M.",
    "N.", "O.", "P.", "Q.", "R.", "S.", "T.", "U.", "V.", "W.", "X.", "Y.", "Z.",
    "1.", "2.", "4.", "4.", "5.", "6.", "7.", "8.", "9.",
    "10.", "20.", "30.", "40.", "50.", "60.", "70.", "80.", "90.",
    "11.", "12.", "13.", "14.", "15.", "16.", "17.", "18.", "19.",
    "21.", "22.", "23.", "24.", "25.", "26.", "27.", "28.", "29.",
    "31.", "32.", "33.", "34.", "35.", "36.", "37.", "38.", "39.",
    "41.", "42.", "43.", "44.", "45.", "46.", "47.", "48.", "49.",
    "51.", "52.", "53.", "54.", "55.", "56.", "57.", "58.", "59.",
    "61.", "62.", "63.", "64.", "65.", "66.", "67.", "68.", "69.",
    "71.", "72.", "73.", "74.", "75.", "76.", "77.", "78.", "79.",
    "81.", "82.", "83.", "84.", "85.", "86.", "87.", "88.", "89.",
    "91.", "92.", "93.", "94.", "95.", "96.", "97.", "98.", "99.",

    "I.", "II.", "III.", "IV.", "V.", "VI.", "VII.", "VIII.", "IX.", "X.",
    "XI.", "XII.", "XIII.", "XIV.", "XV.", "XVI.", "XVII.", "XVIII.", "XIX.", "XX.",
    "i.", "ii.", "iii.", "iv.", "v.", "vi.", "vii.", "viii.", "ix.", "x.",
    "xi.", "xii.", "xiii.", "xiv.", "xv.", "xvi.", "xvii.", "xviii.", "xix.", "xx.",
    "a.", "b.", "c.", "d.", "e.", "f.", "g.", "h.", "i.", "j.", "k.", "l.", "m.",
    "n.", "o.", "p.", "q.", "r.", "s.", "t.", "u.", "v.", "w.", "x.", "y.", "z."
    ]


    # Construct a pattern to match abbreviations
    abbreviations_pattern = r"(?<=\b(?:{})\b)".format("|".join(re.escape(abbr) for abbr in ABBREVIATIONS))

    # This pattern attempts to split at sentence endings (.?!), including the punctuation with the preceding sentence
    # It uses a lookahead to keep the punctuation with the sentence
    # The negative lookbehind (?<!{abbreviations_pattern}) excludes known abbreviations from being split
    sentence_end_regex = r'(?<!{abbreviations_pattern})(?<=[.!?])\s+(?=[A-Z])'.format(abbreviations_pattern=abbreviations_pattern)

    split_sentences_and_punctuation_list = re.split(sentence_end_regex, text)

    # Optionally, remove empty strings if they are not desired
    split_sentences_and_punctuation_list = [s for s in split_sentences_and_punctuation_list if s]

    return split_sentences_and_punctuation_list

def split_sentences_and_punctuation(text):
    """Splits text into sentences, attempting to preserve punctuation and all text content.
    Args:
        text (str): The input text.
    Returns:
        list: A list of sentences with preserved punctuation.


    ABBREVIATIONS
    """
    ABBREVIATIONS = [
    "Dr.", "Mr.", "Mrs.", "Ms.", "Lt.", "St.", "Capt.", "Col.", "Gen.", "Rev.", "Hon.",
    "Jr.", "Sr.", "Ph.D.", "M.D.", "B.A.", "M.A.", "D.D.S.", "J.D.", "M.B.A.", "C.P.A.",
    "a.m.", "p.m.", "etc.", "i.e.", "e.g.", "vs.", "et al.", "approx.", "circa", "fig.",
    "Jan.", "Feb.", "Mar.", "Apr.", "May.", "Jun.", "Jul.", "Aug.", "Sep.", "Oct.", "Nov.", "Dec.",
    "Prof.", "Assoc.", "Asst.", "Dept.", "Univ.", "Gov.", "Sen.", "Rep.", "Pres.",
    "Vol.", "No.", "Pt.", "Ch.", "Sec.", "Subsec.", "Para.", "Subpara.", "Cl.", "Subcl.",
    "Ex.", "Exs.", "Fig.", "Figs.", "Eq.", "Eqs.", "Thm.", "Thms.", "Prop.", "Props.",
    "Def.", "Defs.", "Cor.", "Cors.", "Lem.", "Lems.", "Proof.", "Proofs.",
    "min.", "hrs.", "sec.", "yr.", "yrs.", "mo.", "mos.", "wk.", "wks.",
    "ft.", "in.", "cm.", "mm.", "kg.", "g.", "oz.", "lb.", "lbs.", "ml.", "l.",
    "A.", "B.", "C.", "D.", "E.", "F.", "G.", "H.", "I.", "J.", "K.", "L.", "M.",
    "N.", "O.", "P.", "Q.", "R.", "S.", "T.", "U.", "V.", "W.", "X.", "Y.", "Z.",
    "1.", "2.", "4.", "4.", "5.", "6.", "7.", "8.", "9.",
    "10.", "20.", "30.", "40.", "50.", "60.", "70.", "80.", "90.",
    "11.", "12.", "13.", "14.", "15.", "16.", "17.", "18.", "19.",
    "21.", "22.", "23.", "24.", "25.", "26.", "27.", "28.", "29.",
    "31.", "32.", "33.", "34.", "35.", "36.", "37.", "38.", "39.",
    "41.", "42.", "43.", "44.", "45.", "46.", "47.", "48.", "49.",
    "51.", "52.", "53.", "54.", "55.", "56.", "57.", "58.", "59.",
    "61.", "62.", "63.", "64.", "65.", "66.", "67.", "68.", "69.",
    "71.", "72.", "73.", "74.", "75.", "76.", "77.", "78.", "79.",
    "81.", "82.", "83.", "84.", "85.", "86.", "87.", "88.", "89.",
    "91.", "92.", "93.", "94.", "95.", "96.", "97.", "98.", "99.",

    "I.", "II.", "III.", "IV.", "V.", "VI.", "VII.", "VIII.", "IX.", "X.",
    "XI.", "XII.", "XIII.", "XIV.", "XV.", "XVI.", "XVII.", "XVIII.", "XIX.", "XX.",
    "i.", "ii.", "iii.", "iv.", "v.", "vi.", "vii.", "viii.", "ix.", "x.",
    "xi.", "xii.", "xiii.", "xiv.", "xv.", "xvi.", "xvii.", "xviii.", "xix.", "xx.",
    "a.", "b.", "c.", "d.", "e.", "f.", "g.", "h.", "i.", "j.", "k.", "l.", "m.",
    "n.", "o.", "p.", "q.", "r.", "s.", "t.", "u.", "v.", "w.", "x.", "y.", "z."
    ]


    # Construct a pattern to match abbreviations
    abbreviations_pattern = r"|".join(r"\b{}\b".format(re.escape(abbr)) for abbr in ABBREVIATIONS)

    # This pattern attempts to split at sentence endings (.?!), including the punctuation with the preceding sentence
    # It uses a lookahead to keep the punctuation with the sentence
    # The negative lookahead (?!({abbreviations_pattern})\s) excludes known abbreviations from being split
    sentence_end_regex = r'(?<=[.!?])\s+(?=[A-Z])(?!({abbreviations_pattern})\s)'.format(abbreviations_pattern=abbreviations_pattern)

    split_sentences_and_punctuation_list = re.split(sentence_end_regex, text)

    # Optionally, remove empty strings if they are not desired
    split_sentences_and_punctuation_list = [s for s in split_sentences_and_punctuation_list if s]

    return split_sentences_and_punctuation_list



# def split_sentences_and_punctuation(text):
#     """Splits text into sentences, attempting to preserve punctuation and all text content.

#     Args:
#         text (str): The input text.

#     Returns:
#         list: A list of sentences with preserved punctuation.
#     """

#     # This pattern attempts to split at sentence endings (.?!), including the punctuation with the preceding sentence
#     # It uses a lookahead to keep the punctuation with the sentence
#     sentence_end_regex = r'(?<=[.!?])\s+(?=[A-Z])'

#     split_sentences_and_punctuation_list = re.split(sentence_end_regex, text)

#     # Optionally, remove empty strings if they are not desired
#     split_sentences_and_punctuation_list = [s for s in split_sentences_and_punctuation_list if s]

#     # print("split_sentences_and_punctuation_list")
#     # print(split_sentences_and_punctuation_list)

#     return split_sentences_and_punctuation_list


def recombine_punctuation(sentences):
    """
    A helper function a that
    Recombines floating punctuation and
    creates a new list of sentences."""
    recombined_sentences = []
    i = 0

    while i < len(sentences) - 1:
        # print(i)
        # print(sentences[i-1])
        # print(sentences[i])
        # print(sentences[i+1])

        sentence = sentences[i].strip()
        next_item = sentences[i + 1].strip()

        # if next_item in ".?!":
        if re.match(r"[.?!]+", next_item):
            recombined_sentences.append(sentence + next_item)
            i += 1  # Skip the punctuation since it's been combined
        else:
            recombined_sentences.append(sentence)
        i += 1

    # Add the last sentence (if it exists)
    if sentences[-1]:
        recombined_sentences.append(sentences[-1].strip())

    return recombined_sentences


def chunk_text(sentences, chunk_size, overlap_size=0):
    chunked_text = []
    current_chunk = ""
    overlap_text = ""

    for sentence in sentences:
        # Case 1: Chunk + sentence easily fit
        if len(current_chunk) + len(sentence) + 1 <= chunk_size:
            current_chunk += sentence + " "

        # Case 2: Sentence itself is too big
        elif len(sentence) > chunk_size:
            # Split long sentence (implement 'split_long_sentence' below)
            for sub_sentence in split_long_sentence(sentence, chunk_size):
                chunked_text.append(sub_sentence.strip())

        # Case 3:  Chunk + sentence exceed limit, time to split
        else:
            chunked_text.append(current_chunk.strip())
            current_chunk = sentence + " "

    # Handle final chunk
    if current_chunk:
        chunked_text.append(current_chunk.strip())

    return chunked_text


def split_long_sentence(sentence, chunk_size):
    """Splits a long sentence into chunks, aiming near the  chunk_size."""
    words = sentence.split()
    chunks = []
    current_chunk = ""

    for word in words:
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += word + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = word + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


def check_for_not(chunk, window_size=25):
    """Checks if 'not' is isolated near a potential cut.

    Args:
        chunk (list): A list of sentences forming the chunk.
        window_size (int): The number of characters to consider on either side.

    Returns:
        bool: True if 'not' is isolated, False otherwise.
    """

    joined_chunk = ' '.join(chunk)
    not_indices = [m.start() for m in re.finditer(r'\bnot\b', joined_chunk)]

    for index in not_indices:
        start = max(0, index - window_size)
        end = min(len(joined_chunk), index + window_size)
        if not re.search(r'\w', joined_chunk[start:end]):  # Check for surrounding words
            return True

    return False


def make_chunk_list(text, chunk_size, this_epub_output_dir_path):
    split_sentences_list = split_sentences_and_punctuation(text)
    chunk_list = chunk_text(split_sentences_list, chunk_size)

    for i in chunk_list:
        if not i:
            print_and_log("error None in chunk_list: make_chunk_list()")

    print_and_log(f"len chunk list -> {len(chunk_list)}", this_epub_output_dir_path)

    return chunk_list


######
# Run
######
"""
1. add your epub files into the same current working directory as this script
2. run script
3. find the files in new folders per epub
"""
import glob

# Get the current working directory.
cwd = os.getcwd()

# Search for all EPUB files in the current working directory.
epub_files = glob.glob(os.path.join(cwd, "*.epub"))

# Print the list of EPUB files.
print(epub_files)


####################
# run for each epub
####################
for this_epub_file in epub_files:

    # set target epub to first epub doc listed as being in the cwd
    epub_file_path = this_epub_file

    # make directory for this book
    this_epub_output_dir_path = epub_file_path[:-5] + "_epub_folder"
    print(this_epub_output_dir_path)

    # Set the absolute path
    this_epub_output_dir_path = os.path.abspath(this_epub_output_dir_path)

    # Create a directory for individual txt files
    if not os.path.exists(this_epub_output_dir_path):
        os.makedirs(this_epub_output_dir_path)

    # json
    # output_jsonl_path = 'output.jsonl'
    output_jsonl_path = os.path.join(this_epub_output_dir_path, 'output.jsonl')
    output_json_dir = os.path.join(this_epub_output_dir_path, 'individual_jsons')  # Directory to store individual JSON files
    output_json_zip_dir = os.path.join(this_epub_output_dir_path, 'jsons_zip_archive')  # Directory to store individual JSON files

    # txt
    output_whole_txt_path = os.path.join(this_epub_output_dir_path, 'whole.txt')
    output_txt_dir = os.path.join(this_epub_output_dir_path, 'individual_txt')  # Directory to store individual txt files
    output_txt_zip_dir = os.path.join(this_epub_output_dir_path, 'txt_zip_archive')  # Directory to store individual JSON files


    # chunks
    output_chunks_jsonl_path = os.path.join(this_epub_output_dir_path, 'chunks_jsonl_all.jsonl')  # Directory to store individual txt files
    output_chunks_dir = os.path.join(this_epub_output_dir_path, 'chunk_text_files')  # Directory to store individual txt files
    output_chunks_zip_dir = os.path.join(this_epub_output_dir_path, 'chunks_zip_archive')  # Directory to store individual JSON files

    extract_text_from_epub(epub_file_path,
                           this_epub_output_dir_path,
                           output_jsonl_path,
                           output_json_dir,
                           output_whole_txt_path,
                           output_txt_dir,
                           output_chunks_jsonl_path,
                           output_chunks_dir,
                           max_chunk_size=500)


    # Call the zip function
    """
    zip_folder(path_to_directory_to_zip, output_destination_zip_file_path)
    """
    zip_folder(output_json_dir, output_json_zip_dir)
    zip_folder(output_txt_dir, output_txt_zip_dir)
    zip_folder(output_chunks_dir, output_chunks_zip_dir)



######
# Run
######
"""
1. add your epub files into the same current working directory as this script
2. run script
3. find the files in new folders per epub
"""
import glob

# Get the current working directory.
cwd = os.getcwd()

# Search for all EPUB files in the current working directory.
txt_files = glob.glob(os.path.join(cwd, "*.txt"))

# Print the list of txt files.
print(txt_files)


####################
# run for each txt
####################
for this_txt_file in txt_files:

    # set target txt to first txt doc listed as being in the cwd
    txt_file_path = this_txt_file

    # make directory for this book
    this_txt_output_dir_path = txt_file_path[:-5] + "_txt_folder"
    print(this_txt_output_dir_path)

    # Set the absolute path
    this_txt_output_dir_path = os.path.abspath(this_txt_output_dir_path)

    # Create a directory for individual txt files
    if not os.path.exists(this_txt_output_dir_path):
        os.makedirs(this_txt_output_dir_path)

    # json
    # output_jsonl_path = 'output.jsonl'
    output_jsonl_path = os.path.join(this_txt_output_dir_path, 'output.jsonl')
    output_json_dir = os.path.join(this_txt_output_dir_path, 'individual_jsons')  # Directory to store individual JSON files
    output_json_zip_dir = os.path.join(this_txt_output_dir_path, 'jsons_zip_archive')  # Directory to store individual JSON files

    # txt
    output_whole_txt_path = os.path.join(this_txt_output_dir_path, 'whole.txt')
    output_txt_dir = os.path.join(this_txt_output_dir_path, 'individual_txt')  # Directory to store individual txt files
    output_txt_zip_dir = os.path.join(this_txt_output_dir_path, 'txt_zip_archive')  # Directory to store individual JSON files


    # chunks
    output_chunks_jsonl_path = os.path.join(this_txt_output_dir_path, 'chunks_jsonl_all.jsonl')  # Directory to store individual txt files
    output_chunks_dir = os.path.join(this_txt_output_dir_path, 'chunk_text_files')  # Directory to store individual txt files
    output_chunks_zip_dir = os.path.join(this_txt_output_dir_path, 'chunks_zip_archive')  # Directory to store individual JSON files

    extract_text_from_txt(txt_file_path,
                           this_txt_output_dir_path,
                           output_jsonl_path,
                           output_json_dir,
                           output_whole_txt_path,
                           output_txt_dir,
                           output_chunks_jsonl_path,
                           output_chunks_dir,
                           max_chunk_size=500)


    # Call the zip function
    """
    zip_folder(path_to_directory_to_zip, output_destination_zip_file_path)
    """
    zip_folder(output_json_dir, output_json_zip_dir)
    zip_folder(output_txt_dir, output_txt_zip_dir)
    zip_folder(output_chunks_dir, output_chunks_zip_dir)


[]
['/content/AI_Driven_Entrepreneur_2.txt']
/content/AI_Driven_Entrepreneur__txt_folder
len chunk list -> 345
Size Check, OK \o/
Chunked: split into this many chunks-> 345
/content/AI_Driven_Entrepreneur_2.txt -> ok!
OK!


# Loading dataset as huggingface data

In [ ]:
from datasets import load_dataset, Features, Value

# Define the features of your dataset
features = Features({
    'source_name': Value('string'),
    'text': Value('string')
})

# Load the dataset with the defined features
dataset = load_dataset(
    "json",
    data_files="rustforrustaceans_epub_folder/chunks_jsonl_all.json",  # Ensure this path is correct
    features=features
)

print(dataset)
